In [51]:
import numpy as np
import cupy as cp
from matplotlib import pyplot as plt

use_gpu = False  # Cambiar a False si no tienes GPU o CuPy instalado
array_lib = cp if use_gpu else np

# Consideraciones inciales.

### Funcion para construir la grilla inicial

In [134]:
# Función que construye la grilla inicial

def create_grid(d, grid_size, m, pattern='random'):
    """
    Crea una grilla n-dimensional
    
    Parámetros:
    - d: número de dimensiones
    - grid_size: tamaño de cada dimensión (int o lista de int)
    - m: valor máximo de estado (estados van de 0 a m + 1)
    - pattern: tipo de patrón inicial ('random', 'central', etc.)
    """
    # Convertir grid_size a una tupla si es un solo número
    if isinstance(grid_size, int):
        grid_size = tuple([grid_size] * d)
    else:
        grid_size = tuple(grid_size)
    
    # Crear la grilla según el patrón solicitado
    if pattern == 'random':
        # Grilla con valores aleatorios entre 0 y m
        return array_lib.random.randint(0, m+1, grid_size)
    
    elif pattern == 'central':
        # Grilla con valor alto en el centro y ceros alrededor
        grid = array_lib.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
        
        # Colocar valor m en el centro
        indices = tuple(slice(c-1, c+2) for c in center)
        grid[indices] = m
        return grid
    
    elif pattern == 'checkerboard':
        # Patrón de tablero de ajedrez
        indices = array_lib.indices(grid_size)
        sum_indices = array_lib.sum(indices, axis=0)
        return (sum_indices % 2) * m
    
    else:
        # Patrón predeterminado: valores 0
        return array_lib.zeros(grid_size, dtype=int)
    


### Valores inciales

In [155]:
d = 3  # 3 dimensiones de la grilla
grid_size = 2  # Tamaño de los arreglos
m = 3  # Estados posibles por casilla
r = 1 # Radio de vecindad
grid = create_grid(d, grid_size, m, pattern='random') # Grilla inicial

t = 10 # Tiempo


In [156]:
grid

array([[[2, 3],
        [0, 0]],

       [[2, 0],
        [1, 2]]], dtype=int32)

In [157]:
indices = np.indices(grid.shape)
indices

array([[[[0, 0],
         [0, 0]],

        [[1, 1],
         [1, 1]]],


       [[[0, 0],
         [1, 1]],

        [[0, 0],
         [1, 1]]],


       [[[0, 1],
         [0, 1]],

        [[0, 1],
         [0, 1]]]])

In [158]:
grid.shape

(2, 2, 2)

### Funciones principales

### Pasos a seguir

1. for exterior -> paso del tiempo, por cada tick hace algo
    
    ¿Que hace? 

    a. Saca las otras celdas que pertenezcan a la vecindad

    b. Suma todos los valores 

    c. Se verifica con el intervalo si se + - o mantiene el valor de la celda
